In [23]:
import Nanonis as nano
import re
import numpy as np

In [12]:
f = '/Users/hunfen/OneDrive/General Files/ゼミー/未命名文件夹/2021-03-15/Grid Spectroscopy017.3ds'
f_1 = '/Users/hunfen/OneDrive/General Files/ゼミー/未命名文件夹/2021-03-03/Grid Spectroscopy002.3ds'

In [33]:
grid = nano.read_file(f)

In [34]:
grid.header

{'Grid dim': (50, 50),
 'Grid settings': {'X_OFFSET': -2.613321e-07,
  'Y_OFFSET': -1.896198e-07,
  'X_RANGE': 3e-08,
  'Y_RANGE': 3e-08,
  'ANGLE': 0.0},
 'Filetype': 'Linear',
 'Sweep Signal': 'Bias (V)',
 '# Parameters (4 byte)': 12,
 'Experiment size (bytes)': 9624,
 'Points': 401,
 'Channels': ['Current (A)',
  'LI Demod 1 X (A)',
  'LI Demod 1 Y (A)',
  'Current [bwd] (A)',
  'LI Demod 1 X [bwd] (A)',
  'LI Demod 1 Y [bwd] (A)'],
 'Delay before measuring (s)': 1.0,
 'Experiment': 'Grid Spectroscopy',
 'Start time': '16.03.2021 14:59:07',
 'End time': '17.03.2021 16:37:34',
 'User': '',
 'Comment': '',
 'Bias>Bias (V)': '50E-3',
 'Bias>Calibration (V/V)': '1E+0',
 'Bias>Offset (V)': '0E+0',
 'Bias Spectroscopy>Sweep Start (V)': '50E-3',
 'Bias Spectroscopy>Sweep End (V)': '-34E-3',
 'Bias Spectroscopy>Num Pixel': '401',
 'Bias Spectroscopy>Z Avg time (s)': '50E-3',
 'Bias Spectroscopy>Z offset (m)': '0E+0',
 'Bias Spectroscopy>1st Settling time (s)': '5E-3',
 'Bias Spectroscopy>Se

In [84]:
def _3ds_header_reader(f_path):
    key = ''
    contents = ''
    raw_header = {}
    with open(f_path, 'rb') as f:
        header_end = False
        while not header_end:
            line = f.readline().decode(encoding='utf-8', errors='replace')
            if re.match(':HEADER_END:', line):
                header_end = True
            else:
                key, contents = line.split('=')
                contents = contents.strip('"\r\n')
            raw_header[key] = contents
    return raw_header

def __3ds_header_reform__(raw_header):
    # table = ['Grid settings', 'Channels']
    parameters = ['Fixed parameters', 'Experiment parameters']
    # spec_info_str = ['Filetype', 'Sweep Signal', 'Experiment', 'Start time', 'End time', 'User', 'Comment']
    spec_info_int = [
        '# Parameters (4 byte)', 'Experiment size (bytes)', 'Points'
    ]
    grid_settings = ['X_OFFSET', 'Y_OFFSET', 'X_RANGE', 'Y_RANGE', 'ANGLE']
    header_dict = {}
    keys = list(raw_header.keys())
    # read header into dict
    for i in range(len(keys)):
        header_dict[keys[i]] = raw_header[keys[i]]
    # spec_info_int
    for j in range(len(spec_info_int)):
        header_dict[spec_info_int[j]] = int(header_dict[spec_info_int[j]])
    # Delay before measuring (s)
    header_dict['Delay before measuring (s)'] = float(
        header_dict['Delay before measuring (s)'])
    # Grid dim
    dims = header_dict['Grid dim'].split('x')
    grid_tuple = (int(dims[0]), int(dims[1]))
    grid_check = True
    for k in (0, 1):
        if dims[k] == 1:
            grid_check = False
    header_dict['Grid dim'] = grid_tuple
    # Grid settings
    if grid_check:
        grid_settings_dict = {}
        grid_settings_ls = header_dict['Grid settings'].split(';')
        for i in range(len(grid_settings_ls)):
            grid_settings_dict[grid_settings[i]] = float(grid_settings_ls[i])
        header_dict['Grid settings'] = grid_settings_dict
    else:
        header_dict['Grid settings'] = None
    
    # Modules
    module_keys = []
    module_parameter_keys = []
    module_parameter_values = []
    index = []
    for i in range(len(keys)):
        contents = []
        if re.match('.+\>.+', keys[i]):
            contents = re.split('\>', keys[i])
            index.append(i)
            module_keys.append(contents[0])
            module_parameter_keys.append(contents[1])
            module_parameter_values.append(raw_header[keys[i]])
        else:
            continue

    module_keys_sort = list(set(module_keys))
    module_dict = {}

    for i in range(len(module_keys_sort)):
        module_parameter_dict = {}
        for j in range(len(module_keys)):
            if module_keys[j] == module_keys_sort[i]:
                module_parameter_dict.update({module_parameter_keys[j] : module_parameter_values[j]})
            else:
                continue
        module_dict.update({module_keys_sort[i] : module_parameter_dict})
    for k in range(len(index)):
        del header_dict[keys[index[k]]]
    header_dict['Modules'] = module_dict

    # Parameters
    fixed_parameter = header_dict['Fixed parameters'].split(';')
    experiment_parameters = header_dict['Experiment parameters'].split(';')
    Parameter = []
    for i in range(len(fixed_parameter)):
        Parameter.append(fixed_parameter[i])
    for j in range(len(experiment_parameters)):
        Parameter.append(experiment_parameters[j])
    header_dict['Parameters'] = Parameter
    for k in parameters:
        del header_dict[k]
    # Channels
    header_dict['Channels'] = header_dict['Channels'].split(';')
    # Number of channels
    header_dict['num_Channels'] = len(header_dict['Channels'])
    # self.header = header_dict
    return header_dict

def __3ds_data_reader__(f_path, header):
    """
        __3ds_data_reader__:
        read the .3ds file
        
        Parameters
        ----------
        f_path : path of .3ds file
        header: reformed header of .3ds file
        
        Return
        ------
        Parameters : values of parameters of every position, return (position, num_parameters) np.array
        data : specscopies, returns (position, channels, points) np.array
        """
    with open(f_path, 'rb') as f:
        read_all = f.read()
        offset = read_all.find(':HEADER_END:\x0d\x0a'.encode(encoding='utf-8'))
        # print('found start at {}'.format(offset))
        f.seek(offset + 14)
        data = np.fromfile(f, dtype='>f')
    Parameters = np.zeros((header['Grid dim'][0] * header['Grid dim'][1],
                           header['# Parameters (4 byte)']))
    spec_data = np.zeros((header['Grid dim'][0] * header['Grid dim'][1],
                          header['num_Channels'], header['Points']))
    for i in range(header['Grid dim'][0] * header['Grid dim'][1]):
        # Read Parameters
        for j in range(header['# Parameters (4 byte)']):
            Parameters[i][j] = data[
                i * int(header['# Parameters (4 byte)'] +
                        header['Experiment size (bytes)'] / 4) + j]
        # Read spec data
        for k in range(header['num_Channels']):
            for l in range(header['Points']):
                spec_data[i][k][l] = data[int(
                    i * (header['Experiment size (bytes)'] / 4 +
                         header['# Parameters (4 byte)']) +
                    (k * header['Points'] + header['# Parameters (4 byte)']) +
                    l)]
    # self.Parameters = Parameters
    # self.data = spec_data
    return Parameters, spec_data

In [85]:
raw_header = _3ds_header_reader(f)
header = __3ds_header_reform__(raw_header)

In [86]:
header

{'Grid dim': (50, 50),
 'Grid settings': {'X_OFFSET': -2.613321e-07,
  'Y_OFFSET': -1.896198e-07,
  'X_RANGE': 3e-08,
  'Y_RANGE': 3e-08,
  'ANGLE': 0.0},
 'Filetype': 'Linear',
 'Sweep Signal': 'Bias (V)',
 '# Parameters (4 byte)': 12,
 'Experiment size (bytes)': 9624,
 'Points': 401,
 'Channels': ['Current (A)',
  'LI Demod 1 X (A)',
  'LI Demod 1 Y (A)',
  'Current [bwd] (A)',
  'LI Demod 1 X [bwd] (A)',
  'LI Demod 1 Y [bwd] (A)'],
 'Delay before measuring (s)': 1.0,
 'Experiment': 'Grid Spectroscopy',
 'Start time': '16.03.2021 14:59:07',
 'End time': '17.03.2021 16:37:34',
 'User': '',
 'Comment': '',
 'Modules': {'Lock-in': {'Lock-in status': 'OFF',
   'Modulated signal': 'Bias (V)',
   'Frequency (Hz)': '1.093E+3',
   'Amplitude': '1E-3',
   'Demodulated signal': 'Current (A)',
   'HP Filter Cutoff D1 (Hz)': '303.564E-3',
   'HP Filter Cutoff D2 (Hz)': '9.71405E+0',
   'HP Filter Order D1': '1',
   'HP Filter Order D2': 'OFF',
   'Harmonic D1': '1',
   'Harmonic D2': '2',
   'R

In [80]:
a = '{} > {}'.format(module_keys[1], module_parameter_keys[1])
a

'Bias > Calibration (V/V)'

In [81]:
raw_header[a]

KeyError: 'Bias > Calibration (V/V)'

In [37]:
s = 'Z-Controller>Switch off delay (s)'
if re.match('.+\>.+', s):
    a = re.split('\>', s)

In [39]:
keys = list(raw_header.keys())

In [64]:
module_keys = []
module_parameter_keys = []
module_parameter_values = []
for i in range(len(keys)):
    contents = []
    if re.match('.+\>.+', keys[i]):
        contents = re.split('\>', keys[i])
        module_keys.append(contents[0])
        module_parameter_keys.append(contents[1])
        module_parameter_values.append(raw_header[keys[i]])
    else:
        continue

module_keys_sort = list(set(module_keys))
module_dict = {}

for i in range(len(module_keys_sort)):
    module_parameter_dict = {}
    for j in range(len(module_keys)):
        if module_keys[j] == module_keys_sort[i]:
            module_parameter_dict.update({module_parameter_keys[j] : module_parameter_values[j]})
        else:
            continue
    module_dict.update({module_keys_sort[i] : module_parameter_dict})



In [65]:
module_keys_sort

['Lock-in', 'Z-Controller', 'Current', 'Bias Spectroscopy', 'Bias']

In [71]:
module_dict

{'Lock-in': {'Lock-in status': 'OFF',
  'Modulated signal': 'Bias (V)',
  'Frequency (Hz)': '1.093E+3',
  'Amplitude': '1E-3',
  'Demodulated signal': 'Current (A)',
  'HP Filter Cutoff D1 (Hz)': '303.564E-3',
  'HP Filter Cutoff D2 (Hz)': '9.71405E+0',
  'HP Filter Order D1': '1',
  'HP Filter Order D2': 'OFF',
  'Harmonic D1': '1',
  'Harmonic D2': '2',
  'Reference phase D1 (deg)': '-8.42517E+0',
  'Reference phase D2 (deg)': '0E+0',
  'LP Filter Cutoff D1 (Hz)': '1.2434E+3',
  'LP Filter Cutoff D2 (Hz)': '621.699E+0',
  'LP Filter Order D1': '3',
  'LP Filter Order D2': 'OFF',
  'Sync Filter D1': 'ON',
  'Sync Filter D2': 'OFF'},
 'Z-Controller': {'Z (m)': '-91.0908E-9',
  'Controller name': 'log Current',
  'Controller status': 'ON',
  'Setpoint': '100E-12',
  'Setpoint unit': 'A',
  'P gain': '1.7E-12',
  'I gain': '100E-9',
  'Time const (s)': '17E-6',
  'TipLift (m)': '0E+0',
  'Switch off delay (s)': '300E-3'},
 'Current': {'Current (A)': '104.262E-12',
  'Calibration (A/V)': 